# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [66]:
# Do all imports and installs here
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import gc
import warnings
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
	
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.\
# config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
# .enableHiveSupport().getOrCreate()
# df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [3]:
# #write to parquet
# df_spark.write.parquet("sas_data")
# df_spark=spark.read.parquet("sas_data")

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [70]:
%%time
# Read in the data here
df = pd.DataFrame()
path = r'../data/sas_data/'
for file in os.listdir(path):
    if file.endswith('parquet'):
        df = pd.concat([df, pd.read_parquet(os.path.join(path, file))])

CPU times: user 12.3 s, sys: 5.53 s, total: 17.8 s
Wall time: 13.8 s


In [5]:
df.shape

(3096313, 28)

In [6]:
%%time
# Using Pyarrow
# https://stackoverflow.com/questions/45082832/how-to-read-partitioned-parquet-files-from-s3-using-pyarrow-in-python
dataset = pq.ParquetDataset(path)
table = dataset.read()
py_df = table.to_pandas()

CPU times: user 2.74 s, sys: 2.23 s, total: 4.97 s
Wall time: 1.09 s


In [7]:
py_df.shape

(3096313, 28)

In [8]:
py_df['i94mon'].unique()

array([4.])

In [9]:
del py_df
gc.collect()

218

In [10]:
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,1360825.0,2016.0,4.0,117.0,117.0,NYC,20552.0,1.0,NY,20556.0,66.0,2.0,1.0,20160408,None,None,O,O,None,M,1950.0,07062016,None,None,DL,5.582890e+10,00475,WT
1,1360826.0,2016.0,4.0,117.0,117.0,NYC,20552.0,1.0,NY,20556.0,63.0,2.0,1.0,20160408,None,None,O,O,None,M,1953.0,07062016,None,None,DL,5.582914e+10,00475,WT
2,1360827.0,2016.0,4.0,117.0,117.0,NYC,20552.0,1.0,NY,20556.0,47.0,2.0,1.0,20160408,None,None,O,O,None,M,1969.0,07062016,None,None,EK,5.583591e+10,00205,WT
3,1360828.0,2016.0,4.0,117.0,117.0,NYC,20552.0,1.0,NY,20556.0,44.0,2.0,1.0,20160408,None,None,O,O,None,M,1972.0,07062016,None,None,AA,5.580522e+10,00199,WT
4,1360829.0,2016.0,4.0,117.0,117.0,NYC,20552.0,1.0,NY,20556.0,42.0,2.0,1.0,20160408,None,None,O,O,None,M,1974.0,07062016,M,None,JJ,7.185010e+08,8078,WT


In [11]:
df.shape

(3096313, 28)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3096313 entries, 0 to 220159
Data columns (total 28 columns):
 #   Column    Dtype  
---  ------    -----  
 0   cicid     float64
 1   i94yr     float64
 2   i94mon    float64
 3   i94cit    float64
 4   i94res    float64
 5   i94port   object 
 6   arrdate   float64
 7   i94mode   float64
 8   i94addr   object 
 9   depdate   float64
 10  i94bir    float64
 11  i94visa   float64
 12  count     float64
 13  dtadfile  object 
 14  visapost  object 
 15  occup     object 
 16  entdepa   object 
 17  entdepd   object 
 18  entdepu   object 
 19  matflag   object 
 20  biryear   float64
 21  dtaddto   object 
 22  gender    object 
 23  insnum    object 
 24  airline   object 
 25  admnum    float64
 26  fltno     object 
 27  visatype  object 
dtypes: float64(13), object(15)
memory usage: 685.1+ MB


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data
1) Change datatypes of several fields. Most of the floats can be converted to ints/str

In [13]:
# Performing cleaning tasks here
float_dtypes = df.select_dtypes('float').columns
for col in float_dtypes:
    # if col == 'arrdate' or col == 'depdate': continue
    df[col] = df[col].astype(pd.Int32Dtype(), errors='ignore')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3096313 entries, 0 to 220159
Data columns (total 28 columns):
 #   Column    Dtype  
---  ------    -----  
 0   cicid     Int32  
 1   i94yr     Int32  
 2   i94mon    Int32  
 3   i94cit    Int32  
 4   i94res    Int32  
 5   i94port   object 
 6   arrdate   Int32  
 7   i94mode   Int32  
 8   i94addr   object 
 9   depdate   Int32  
 10  i94bir    Int32  
 11  i94visa   Int32  
 12  count     Int32  
 13  dtadfile  object 
 14  visapost  object 
 15  occup     object 
 16  entdepa   object 
 17  entdepd   object 
 18  entdepu   object 
 19  matflag   object 
 20  biryear   Int32  
 21  dtaddto   object 
 22  gender    object 
 23  insnum    object 
 24  airline   object 
 25  admnum    float64
 26  fltno     object 
 27  visatype  object 
dtypes: Int32(12), float64(1), object(15)
memory usage: 578.8+ MB


In [15]:
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,1360825,2016,4,117,117,NYC,20552,1,NY,20556,66,2,1,20160408,None,None,O,O,None,M,1950,07062016,None,None,DL,5.582890e+10,00475,WT
1,1360826,2016,4,117,117,NYC,20552,1,NY,20556,63,2,1,20160408,None,None,O,O,None,M,1953,07062016,None,None,DL,5.582914e+10,00475,WT
2,1360827,2016,4,117,117,NYC,20552,1,NY,20556,47,2,1,20160408,None,None,O,O,None,M,1969,07062016,None,None,EK,5.583591e+10,00205,WT
3,1360828,2016,4,117,117,NYC,20552,1,NY,20556,44,2,1,20160408,None,None,O,O,None,M,1972,07062016,None,None,AA,5.580522e+10,00199,WT
4,1360829,2016,4,117,117,NYC,20552,1,NY,20556,42,2,1,20160408,None,None,O,O,None,M,1974,07062016,M,None,JJ,7.185010e+08,8078,WT


In [16]:
df[df.duplicated()].shape

(0, 28)

In [17]:
# Possible duplicate admissions
df['admnum'].nunique()

3075579

In [18]:
# Number of duplicates based on admission number
df[df['admnum'].duplicated()].shape

(20734, 28)

In [19]:
df[df['admnum'].duplicated()]['admnum'].value_counts()

0.000000e+00    67
7.812055e+10    10
4.652077e+10     8
8.924999e+10     8
4.701041e+10     7
                ..
2.908534e+09     1
5.605863e+10     1
5.582277e+10     1
4.228673e+10     1
5.581274e+10     1
Name: admnum, Length: 19374, dtype: int64

In [20]:
df[df['arrdate'] == 'D/S']

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype


In [67]:
sas_date_converter = lambda x: None if x!= x else datetime.strptime('1960-01-01', "%Y-%m-%d") + timedelta(x)

In [59]:
def date_change(row):
    if row is None:
        return None
    if row.endswith('2016'):
        return datetime.strptime(row, '%m%d%Y')
    if row.startswith('2016'):
        return datetime.strptime(row, '%Y%m%d')
    return None

In [60]:
#df[df['dtaddto'].notnull()]['dtaddto'].apply(lambda x: datetime.strptime(x, '%Y%m%d') if x.startswith('2016') else None)

#df['dtaddto'].sample(50).apply(date_change)


In [37]:
df[df['dtaddto'].isnull()].shape

(477, 28)

In [21]:
data_missing = pd.Series(df.isnull().sum()/df.shape[0] * 100).\
to_frame().\
rename(columns={0: 'pct_missing'}).\
reset_index().\
rename(columns={'index': 'field'})
data_missing[data_missing['pct_missing'] > 0].sort_values('pct_missing', ascending=False)

,field,pct_missing
18,entdepu,99.987340
15,occup,99.737559
23,insnum,96.327632
14,visapost,60.757746
22,gender,13.379429
8,i94addr,4.928184
9,depdate,4.600859
17,entdepd,4.470769
19,matflag,4.470769
24,airline,2.700857


Note

The entdepu, occup and insnum fields can be moved out of this DataFrame while building
the data model

In [22]:
# Distribution month - Needs confirmation - Only for Apr?
df['i94mon'].value_counts()

4    3096313
Name: i94mon, dtype: Int64

In [23]:
# popular cities
df['i94cit'].value_counts().head(5)

135    360157
209    206873
245    191425
111    188766
582    175781
Name: i94cit, dtype: Int64

In [24]:
# popular res
df['i94res'].value_counts().head(5)

135    368421
209    249167
245    185609
111    185339
582    179603
Name: i94res, dtype: Int64

In [25]:
# popular port
df['i94port'].value_counts().head(5)

NYC    485916
MIA    343941
LOS    310163
SFR    152586
ORL    149195
Name: i94port, dtype: int64

In [26]:
# arrdate - Number of days since 1960-01-01
df['arrdate'] = pd.to_timedelta(df['arrdate'], unit='d') + pd.datetime(1960, 1, 1)

In [27]:
df['arrdate'].min(), df['arrdate'].max()

(Timestamp('2016-04-01 00:00:00'), Timestamp('2016-04-30 00:00:00'))

In [28]:
# mode of travel
df['i94mode'].value_counts(dropna=False)

1      2994505
3        66660
2        26349
9         8560
NaN        239
Name: i94mode, dtype: Int64

In [76]:
# address
df['i94addr'].value_counts(dropna=False).head()

FL     621701
NY     553677
CA     470386
HI     168764
NaN    152592
Name: i94addr, dtype: int64

In [77]:
df['depdate'] = pd.to_timedelta(df['depdate'], unit='d') + pd.datetime(1960, 1, 1)

In [78]:
df['depdate'].min(), df['depdate'].max()

(Timestamp('2001-07-20 00:00:00'), Timestamp('2084-05-16 00:00:00'))

In [80]:
# Age of respondent years
df['i94bir'].value_counts(dropna=False).head()

30    71958
33    70415
31    70409
34    70251
32    69809
Name: i94bir, dtype: Int64

In [81]:
# Visa
df['i94visa'].value_counts(dropna=False)

2      2530868
1       522079
3        43366
NaN          0
Name: i94visa, dtype: Int64

In [82]:
# count - Used for summary stats. Can be removed during modeling
df['count'].value_counts(dropna=False)

1      3096313
NaN          0
Name: count, dtype: Int64

In [ ]:
# Fields not used by CIC can be dropped
dtadfile, visapost, occup, entdepa, entdepd, entdepu, dtaddto

In [83]:
# Match flag
df['matflag'].value_counts(dropna=False)

M      2957884
NaN     138429
Name: matflag, dtype: int64

In [85]:
# Birth year
df['biryear'].value_counts(dropna=False).head()

1986    71958
1983    70415
1985    70409
1982    70251
1984    69809
Name: biryear, dtype: Int64

In [86]:
# Gender
df['gender'].value_counts(dropna=False)

M      1377224
F      1302743
NaN     414269
X         1610
U          467
Name: gender, dtype: int64

In [87]:
# Airline
df['airline'].value_counts(dropna=False).head()

AA    310091
UA    264271
DL    252526
BA    190997
LH    120556
Name: airline, dtype: int64

In [88]:
# Visa type
df['visatype'].value_counts(dropna=False)

WT     1309059
B2     1117897
WB      282983
B1      212410
GMT      89133
F1       39016
E2       19383
CP       14758
E1        3743
I         3176
F2        2984
M1        1317
I1         234
GMB        150
M2          49
SBP         11
CPL         10
Name: visatype, dtype: int64

In [71]:
df['arrdate'].max()

20574.0

In [72]:
df['arrdate'].apply(sas_date_converter).head()

0   2016-04-08
1   2016-04-08
2   2016-04-08
3   2016-04-08
4   2016-04-08
Name: arrdate, dtype: datetime64[ns]

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

With Immigrations, the data can be broken into:
    
1) Passenger<br/>
2) Port reference code<br/>
3) Immigrations<br/>
4) Airline reference code<br/>
5) Mode of arrival reference code<br/>
6) Visa type reference code<br/>

Passenger
- Admission, Year, Gender, Occupation

Immigrations
- cicid, i94cit, i94res, i94port


In [94]:
!ls

Capstone Project Template.ipynb


In [99]:
demo_file = r'../data/us-cities-demographics.csv'
demo_df = pd.read_csv(demo_file, sep=';')

In [100]:
demo_df.shape

(2891, 12)

In [101]:
demo_df.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [102]:
demo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   City                    2891 non-null   object 
 1   State                   2891 non-null   object 
 2   Median Age              2891 non-null   float64
 3   Male Population         2888 non-null   float64
 4   Female Population       2888 non-null   float64
 5   Total Population        2891 non-null   int64  
 6   Number of Veterans      2878 non-null   float64
 7   Foreign-born            2878 non-null   float64
 8   Average Household Size  2875 non-null   float64
 9   State Code              2891 non-null   object 
 10  Race                    2891 non-null   object 
 11  Count                   2891 non-null   int64  
dtypes: float64(6), int64(2), object(4)
memory usage: 271.2+ KB


In [103]:
# change data types of floats to ints
float_dtypes = demo_df.select_dtypes('float').columns
for col in float_dtypes:
    # if col == 'arrdate' or col == 'depdate': continue
    demo_df[col] = demo_df[col].astype(pd.Int32Dtype(), errors='ignore')

In [104]:
demo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   City                    2891 non-null   object 
 1   State                   2891 non-null   object 
 2   Median Age              2891 non-null   float64
 3   Male Population         2888 non-null   Int32  
 4   Female Population       2888 non-null   Int32  
 5   Total Population        2891 non-null   int64  
 6   Number of Veterans      2878 non-null   Int32  
 7   Foreign-born            2878 non-null   Int32  
 8   Average Household Size  2875 non-null   float64
 9   State Code              2891 non-null   object 
 10  Race                    2891 non-null   object 
 11  Count                   2891 non-null   int64  
dtypes: Int32(4), float64(2), int64(2), object(4)
memory usage: 237.3+ KB


In [105]:
demo_df.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129,49500,93629,4147,32935,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040,46799,84839,4819,8229,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127,87105,175232,5821,33878,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040,143873,281913,5829,86253,2.73,NJ,White,76402


In [106]:
demo_df[demo_df.duplicated()]

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count


In [107]:
demo_data_missing = pd.Series(demo_df.isnull().sum()/demo_df.shape[0] * 100).\
to_frame().\
rename(columns={0: 'pct_missing'}).\
reset_index().\
rename(columns={'index': 'field'})
demo_data_missing[demo_data_missing['pct_missing'] > 0].sort_values('pct_missing', ascending=False)

,field,pct_missing
8,Average Household Size,0.553442
6,Number of Veterans,0.449671
7,Foreign-born,0.449671
3,Male Population,0.103770
4,Female Population,0.103770


In [118]:
demo_df[(demo_df['State'] == 'Maryland') &
       (demo_df['City'] == 'Frederick')].sort_values('City')

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
23,Frederick,Maryland,36.1,33146,36336,69482,3870,14211,2.48,MD,White,49181
816,Frederick,Maryland,36.1,33146,36336,69482,3870,14211,2.48,MD,Black or African-American,13372
1119,Frederick,Maryland,36.1,33146,36336,69482,3870,14211,2.48,MD,Asian,5711
1208,Frederick,Maryland,36.1,33146,36336,69482,3870,14211,2.48,MD,Hispanic or Latino,13065
2602,Frederick,Maryland,36.1,33146,36336,69482,3870,14211,2.48,MD,American Indian and Alaska Native,1333


In [115]:
demo_df[(demo_df['State'] == 'Maryland')]

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.60,MD,Hispanic or Latino,25924
23,Frederick,Maryland,36.1,33146,36336,69482,3870,14211,2.48,MD,White,49181
62,Columbia,Maryland,37.9,52202,51265,103467,6526,23249,2.68,MD,Black or African-American,30075
230,Waldorf,Maryland,33.6,35640,39872,75512,6932,5954,2.69,MD,Hispanic or Latino,4810
259,Ellicott City,Maryland,42.4,34406,36526,70932,3856,17587,2.73,MD,American Indian and Alaska Native,396
287,German,Maryland,34.9,41115,43007,84122,2443,27877,2.95,MD,Hispanic or Latino,18202
444,German,Maryland,34.9,41115,43007,84122,2443,27877,2.95,MD,Asian,15921
461,Rockville,Maryland,38.1,31205,35793,66998,1990,25047,2.60,MD,American Indian and Alaska Native,594
550,Ellicott City,Maryland,42.4,34406,36526,70932,3856,17587,2.73,MD,Asian,22551
592,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.60,MD,White,37756


In [113]:
661388-621849

39539

In [ ]:
Note

Can remove Total Population<br/>
Can divide the data into State and City level info to avoid the duplication of State level info

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.